In [1]:
from transformers import AutoTokenizer, BertForMaskedLM
import torch
from dotenv import load_dotenv
import os
import pandas as pd
from transformers import AdamW
from tqdm import tqdm

load_dotenv()
DATASET_PATH = f'{os.getenv("DATASETS_PATH")}bert_training_data.xlsx'

/home/ksaff/miniconda3/envs/smlp_env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-08-29 23:38:02.900349: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-08-29 23:38:02.922679: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-29 23:38:03.288078: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find

In [2]:
df = pd.read_excel(DATASET_PATH)
text = df[['nlp_2', 'nlp_3', 'nlp_4', 'nlp_5']].values
text = text.flatten()

In [3]:
tokenizer = AutoTokenizer.from_pretrained('allegro/herbert-base-cased')

inputs = tokenizer(list(text), return_tensors='pt', max_length=60, truncation=True, padding='max_length') 
inputs['labels'] = inputs.input_ids.detach().clone()
inputs.input_ids[0]

tensor([    0,  3379,  1028,  2017,  2134, 23389,  1899,  3379,  1028, 37696,
         1899, 24656,  6585,  1899,  2954,  2025,    87,  3953,  2243,  1899,
        47890,  2483,  1899, 13186,  1011,  1899,  3248, 10321,  3304,  1899,
        30926,  4167,  1899,    57,  2009, 14323,  1899, 25503,  1899,     2,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1])

In [4]:
rand = torch.rand(inputs.input_ids.shape)
mask_arr = (rand < 0.15) * (inputs.input_ids != 0) * (inputs.input_ids != 1) * (inputs.input_ids != 2)
mask_arr

tensor([[False,  True, False,  ..., False, False, False],
        [False, False, False,  ..., False, False, False],
        [False, False, False,  ..., False, False, False],
        ...,
        [False, False, False,  ..., False, False, False],
        [False, False, False,  ..., False, False, False],
        [False, False, False,  ..., False, False, False]])

In [5]:
selection = []

for i in range(mask_arr.shape[0]):
    selection.append(torch.flatten(mask_arr[i].nonzero()).tolist())

for i in range(mask_arr.shape[0]):
    inputs.input_ids[i, selection[i]] = 103

inputs.input_ids[0][:30]

tensor([    0,   103,  1028,  2017,  2134, 23389,  1899,  3379,  1028, 37696,
         1899, 24656,  6585,  1899,  2954,  2025,    87,  3953,   103,  1899,
        47890,  2483,  1899, 13186,  1011,  1899,  3248, 10321,  3304,  1899])

In [6]:
class StatmentsDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings
        
    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        
    def __len__(self):
        return len(self.encodings.input_ids)

In [7]:
dataset = StatmentsDataset(inputs)
data_loader = torch.utils.data.DataLoader(dataset, batch_size=16, shuffle=True)

In [8]:
bert_model = BertForMaskedLM.from_pretrained("allegro/herbert-base-cased")

bert_model.train()

BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(50000, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_a

In [9]:
optimizer = AdamW(bert_model.parameters(), lr=1e-3)

epochs = 2

for epoch in range(epochs):
    loop = tqdm(data_loader, leave=True)
    for batch in  loop:
        optimizer.zero_grad()
        input_ids = batch['input_ids']
        attention_mask = batch['attention_mask']
        labels = batch['labels']

        outputs = bert_model(input_ids, attention_mask=attention_mask, labels=labels)

        loss = outputs.loss
        loss.backward()
        optimizer.step()

        loop.set_description(f'Epoch {epoch}')
        loop.set_postfix(loss=loss.item())

/home/ksaff/miniconda3/envs/smlp_env/lib/python3.11/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
  0%|          | 0/131 [00:00<?, ?it/s]/tmp/ipykernel_418740/3942611719.py:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
Epoch 1: 100%|██████████| 131/131 [03:06<00:00,  1.42s/it, loss=2.02]


In [10]:
bert_model.save_pretrained("trial_model")